In [1]:
from pyscf import gto, scf, cc, lib
import numpy as np
from functools import partial
import os

np.einsum = partial(np.einsum, optimize=True)
np.set_printoptions(6, suppress=True, linewidth=150)

## System Definition and Reference CC

In [2]:
# 10.1021/jp104865w, (H2O)10, PP5 structure
mol = gto.Mole(atom="""
    O -2.21165 0.99428 -1.34761
    H -1.39146 1.51606 -1.47747
    H -1.97320 0.08049 -1.61809
    O 0.09403 2.29278 1.59474
    H 0.12603 2.53877 0.64902
    H -0.74393 1.78978 1.67135
    O -1.36387 -1.68942 -1.58413
    H -1.87986 -2.36904 -2.04608
    H -1.51808 -1.85775 -0.60321
    O 1.15753 -1.98493 1.42883
    H 1.51336 -1.05256 1.56992
    H 1.63126 -2.54067 2.06706
    O 2.16234 0.46384 1.59959
    H 1.45220 1.14162 1.73767
    H 2.44819 0.61600 0.67631
    O 0.26320 2.39844 -1.29615
    H 1.04651 1.79827 -1.38236
    H 0.46651 3.18119 -1.83082
    O 1.44377 -1.86519 -1.36370
    H 0.48945 -1.86011 -1.60072
    H 1.44320 -2.10978 -0.41122
    O -1.62831 -1.98091 1.04938
    H -1.92768 -1.08892 1.33229
    H -0.69028 -2.03600 1.33896
    O 2.35473 0.62384 -1.26848
    H 3.15897 0.65726 -1.80967
    H 2.00663 -0.31760 -1.36507
    O -2.29362 0.74293 1.32406
    H -2.34790 0.87628 0.33220
    H -3.13510 1.07144 1.67759
""", basis="cc-pVDZ").build()

In [3]:
mf = scf.RHF(mol).density_fit(auxbasis="cc-pVDZ-ri").run()

converged SCF energy = -760.417170870121


In [4]:
mol.verbose = 4
lib.logger.TIMER_LEVEL = 1

In [6]:
mf_cc = cc.CCSD(mf, frozen=10).run()


******** <class 'pyscf.cc.dfccsd.RCCSD'> ********
CC2 = 0
CCSD nocc = 40, nmo = 230
frozen orbitals 10
max_cycle = 50
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-05
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 4000 MB (current use 450 MB)
    CPU time for df vj and vk      0.55 sec, wall time      0.03 sec
    CPU time for df vj and vk      0.57 sec, wall time      0.04 sec


<class 'pyscf.cc.dfccsd.RCCSD'> does not have attributes  converged


Init t2, MP2 energy = -762.512442320864  E_corr(MP2) -2.09527145074345
    CPU time for init mp2      2.41 sec, wall time      0.41 sec
Init E_corr(RCCSD) = -2.09527145074912
cycle = 1  E_corr(RCCSD) = -2.12370362944668  dE = -0.0284321787  norm(t1,t2) = 0.0841487
    CPU time for RCCSD iter    396.32 sec, wall time     26.83 sec
cycle = 2  E_corr(RCCSD) = -2.16249030073982  dE = -0.0387866713  norm(t1,t2) = 0.0246926
    CPU time for RCCSD iter    439.82 sec, wall time     28.49 sec
cycle = 3  E_corr(RCCSD) = -2.16887833109026  dE = -0.00638803035  norm(t1,t2) = 0.0115431
    CPU time for RCCSD iter    455.28 sec, wall time     29.67 sec
cycle = 4  E_corr(RCCSD) = -2.17377752172915  dE = -0.00489919064  norm(t1,t2) = 0.00379069
    CPU time for RCCSD iter    447.13 sec, wall time     29.23 sec
cycle = 5  E_corr(RCCSD) = -2.17369713739983  dE = 8.03843293e-05  norm(t1,t2) = 0.000948251
    CPU time for RCCSD iter    448.51 sec, wall time     29.29 sec
cycle = 6  E_corr(RCCSD) = -2.1735

## Global Variables

In [7]:
nfrz = 10
cderi_ao = mf.with_df._cderi
mo_occ = mf.mo_occ[nfrz:]
mo_energy = mf.mo_energy[nfrz:]
mo_coeff = mf.mo_coeff[:, nfrz:]

nmo = len(mo_occ)
nocc = (mo_occ > 0).sum()
nvir = nmo - nocc
nao = mol.nao
naux = cderi_ao.shape[0]

so = slice(0, nocc)
sv = slice(nocc, nmo)

In [8]:
B = np.einsum("Puv, up, vq -> pqP", lib.unpack_tril(cderi_ao), mo_coeff, mo_coeff)
D_ov = mo_energy[so, None] - mo_energy[None, sv]

In [12]:
os.makedirs("h2o_pp5-cc-pvdz", exist_ok=True)
np.save("h2o_pp5-cc-pvdz/mo_coeff.npy", np.ascontiguousarray(mo_coeff))
np.save("h2o_pp5-cc-pvdz/mo_occ.npy", mo_occ)
np.save("h2o_pp5-cc-pvdz/mo_energy.npy", mo_energy)
np.save("h2o_pp5-cc-pvdz/cderi.npy", mf.with_df._cderi)